# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedElemRestriction

Here we show some basic examples to illustrate the `libceed.ElemRestriction` class. In libCEED, a `libceed.ElemRestriction` groups the degrees of freedom (dofs) of the local vector according to the different elements they belong to (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

Here we illustrate the simple creation and application of a `libceed.ElemRestriction`, with user provided dof indices

In [ ]:
import libceed
import numpy as np

# In this 1D example, the dofs are indexed as
#
# Restriction input:
#  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13
#
# Restriction output:
#  x --  x |  x --  x | x --  x
# 10 -- 11 | 11 -- 12 | 12 -- 13

ceed = libceed.Ceed()

ne = 3

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1
    
r = ceed.ElemRestriction(ne, 2, 1, 1, ne+1, ind, cmode=libceed.USE_POINTER)

y = ceed.Vector(2*ne)
y.set_value(0)

r.apply(x, y)

with y.array_read() as y_array:
  print('y =', y_array)


* In the following example, we illustrate how to extract the multiplicity of indices in an element restriction

In [ ]:
# In this 1D example, there are four nodes per element
# 
#  x -- o -- o -- x -- o -- o -- x -- o -- o -- x

ne = 3

ind = np.zeros(4*ne, dtype="int32")

for i in range(ne):
  ind[4*i+0] = i*3+0
  ind[4*i+1] = i*3+1
  ind[4*i+2] = i*3+2
  ind[4*i+3] = i*3+3

r = ceed.ElemRestriction(ne, 4, 1, 1, 3*ne+1, ind, cmode=libceed.USE_POINTER)

mult = r.get_multiplicity()

with mult.array_read() as m_array:
  print('mult =', m_array)


* In the following example, we illustrate the creation and use of a strided (identity) element restriction. Strided restrictions are typically used for data stored at quadrature points or for vectors stored in the [E-vector](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#finite-element-operator-decomposition) format, such as in Discontinuous Galerkin approximations.

In [ ]:
# In this 1D example, the dofs are indexed as
#
# Restriction input:
#   x   --   x   --   x
# 10-11 -- 12-13 -- 14-15
#
# Restriction output:
#  x --  x |  x --  x |  x --  x
# 10 -- 11 | 12 -- 13 | 14 -- 15

ne = 3

x = ceed.Vector(2*ne)
a = np.arange(10, 10 + 2*ne, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

strides = np.array([1, 2, 2], dtype="int32")

r = ceed.StridedElemRestriction(ne, 2, 1, 2*ne, strides)

y = ceed.Vector(2*ne)
y.set_value(0)

r.apply(x, y)

with y.array_read() as y_array:
  print('y =', y_array)


* In the following example, we illustrate the creation and view of a blocked strided (identity) element restriction

In [ ]:
# In this 1D example, there are three elements (four nodes in total) 
# 
#  x -- x -- x -- x

ne = 3

strides = np.array([1, 2, 2], dtype="int32")

r = ceed.BlockedStridedElemRestriction(ne, 2, 2, 1, ne+1, strides)

print(r)

### Advanced topics

* In the following example (intended for backend developers), we illustrate the creation of a blocked element restriction (from an L-vector to an E-vector) and its transpose (inverse operation, from an E-vector to an L-vector)

In [ ]:
# In this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x --  x --  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13 -- 14 -- 15 -- 16 -- 17 -- 18
# 
# We block elements into groups of 5:
#  ________________________________________________________________________________________
# |                   block 0:                            |         block 1:               |
# |     e0    |    e1    |    e2    |    e3    |    e4    |    e0    |    e1    |    e2    |
# |                                                       |                                |
# |  x --  x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x  -- x |
# | 10 -- 11  | 11   12  | 12   13  | 13   14  | 14   15  | 15   16  | 16   17  |  17   18 |
#
# Intermediate logical representation:
#  ______________________________________________________________________________
# |               block 0:               |               block 1:               |
# |     node0:                 node1:    |     node0:                 node1:    |
# | 10-11-12-13-14        11-12-13-14-15 | 15-16-17- *- *        16-17-18- *- * |
# | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 |
#
# Forward restriction output:
#  ______________________________________________________________________________
# |               block 0:               |               block 1:               |
# |     node0:                 node1:    |     node0:                 node1:    |
# | 10-11-12-13-14        11-12-13-14-15 | 15-16-17-17-17        16-17-18-18-18 |
# | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 |

ne = 8
blksize = 5

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1

r = ceed.BlockedElemRestriction(ne, 2, blksize, 1, 1, ne+1, ind,
                                cmode=libceed.USE_POINTER)

y = ceed.Vector(2*blksize*2)
y.set_value(0)

r.apply(x, y)

with y.array_read() as y_array:
  print('y =', y_array)

x.set_value(0)
r.T.apply(y, x)

with x.array_read() as x_array:
  print('x =', x_array)

* In the following example (intended for backend developers), we illustrate the creation and application of a blocked element restriction (from an L-vector to an E-vector) and its transpose (inverse operation, from an E-vector to an L-vector)

In [ ]:
# In this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x --  x --  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13 -- 14 -- 15 -- 16 -- 17 -- 18
#
# We block elements into groups of 5:
#  ________________________________________________________________________________________
# |                   block 0:                            |         block 1:               |
# |     e0    |    e1    |    e2    |    e3    |    e4    |    e0    |    e1    |    e2    |
# |                                                       |                                |
# |  x --  x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x -- x  |  x  -- x |
# | 10 -- 11  | 11   12  | 12   13  | 13   14  | 14   15  | 15   16  | 16   17  |  17   18 |
#
# Intermediate logical representation (extraction of block1 only in this case):
#  _______________________________________
# |               block 1:               |
# |     node0:                 node1:    |
# | 15-16-17- *- *        16-17-18- *- * |
# | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 |
#
# Forward restriction output:
#  _______________________________________
# |               block 1:               |
# |     node0:                 node1:    |
# | 15-16-17-17-17        16-17-18-18-18 |
# | e0 e1 e2 e3 e4        e0 e1 e2 e3 e4 |

ne = 8
blksize = 5

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1

r = ceed.BlockedElemRestriction(ne, 2, blksize, 1, 1, ne+1, ind,
                                cmode=libceed.USE_POINTER)

y = ceed.Vector(blksize*2)
y.set_value(0)

r.apply_block(1, x, y)

with y.array_read() as y_array:
  print('y =', y_array)

x.set_value(0)
r.T.apply_block(1, y, x)

with x.array_read() as array:
  print('x =', x_array)

Note that the nodes at the boundary between elements have multiplicty 2, while the internal nodes and the outer boundary nodes, have multiplicity 1.